In [17]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle as pkl
import sys

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

import torch.utils.data as data 
import torch
from torch_geometric.data import Data
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch.utils.data import Dataset, DataLoader

import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import MessagePassing, SAGEConv
from ogb.nodeproppred import Evaluator #PygNodePropPredDatase

from IPython.display import clear_output


from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.nn.functional as F



In [18]:
# Read files
path = "C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/dades_guillem/"
df_train_init = pd.read_csv(path + "train_set.csv") 
df_test_init = pd.read_csv(path + "test_set.csv")
# df_train_edges = pd.read_csv(path + "train_edges.csv")
# df_test_edges = pd.read_csv(path + "test_edges.csv")

In [19]:
# AIXO JA NO HAURIA DE CALDRE, ESTA AL DATA SPLIT
path = "C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/dades_guillem/"
df_classes = pd.read_csv(path + "elliptic_txs_classes.csv") # Nodes' labels
df_edges_init = pd.read_csv(path + "elliptic_txs_edgelist.csv") # Edges
df_features = pd.read_csv(path + "elliptic_txs_features.csv", header=None) # Nodes' features

# Change column names of df_features
colNames1 = {'0': 'txId', 1: "Time step"}
colNames2 = {str(ii+2): "Local_feature_" + str(ii+1) for ii in range(93)}
colNames3 = {str(ii+95): "Aggregate_feature_" + str(ii+1) for ii in range(72)}

colNames = dict(colNames1, **colNames2, **colNames3 )
colNames = {int(jj): item_kk for jj,item_kk in colNames.items()}

df_features = df_features.rename(columns=colNames)

In [20]:
# AIXO JA NO HAURIA DE CALDRE, ESTA AL DATA SPLIT
def prep_df(feats: pd.DataFrame, edges: pd.DataFrame):
    #1 és la classe illicit, 2 la  licit
    df_feats = feats.loc[feats['class'].isin([1, 2])]
    df_feats['class'] = df_feats['class'].replace({1: 0, 2: 1})
    df_feats = df_feats.reset_index(drop=True)

    #ens quedem només amb els edges que apareixen en el nodes d'entrenament
    df_edges = edges.loc[((edges['txId1'].isin(df_feats['txId'])) & (df_edges_init['txId2'].isin(df_feats['txId'])))]
    df_edges = df_edges.reset_index(drop=True)
    print(f"contador de valors per classe: \n {df_feats['class'].value_counts()}\n")
    return  df_feats, df_edges

df_train, df_edges_train = prep_df(df_train_init, df_edges_init)
df_test, df_edges_test = prep_df(df_test_init, df_edges_init)

contador de valors per classe: 
 class
1    34654
0     2672
Name: count, dtype: int64

contador de valors per classe: 
 class
1    7365
0    1873
Name: count, dtype: int64



C:\Users\gsamp\AppData\Local\Temp\ipykernel_12680\1529718817.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feats['class'] = df_feats['class'].replace({1: 0, 2: 1})


In [21]:
def map_idx(feats: pd.DataFrame, edges: pd.DataFrame, save = True, loading_dir = "a"):
    mapping_txid = dict(zip(feats['txId'], list(feats.index)))
    dir = 'dades_guillem/' + str(loading_dir) + '.pkl'
    if save:
        df_edges_mapped = edges.replace({'txId1': mapping_txid, 'txId2': mapping_txid})
        
        df_edges_mapped.to_pickle(loading_dir)
    else:
        df_edges_mapped = pd.read_pickle(loading_dir)
    return df_edges_mapped

df_edges_mapped_train = map_idx(feats = df_train, edges = df_edges_train, save = True, loading_dir='train')
df_edges_mapped_test = map_idx(feats = df_test, edges = df_edges_test, save = True, loading_dir='test')


In [22]:
def get_data(feats: pd.DataFrame, edges:pd.DataFrame):
    x = torch.tensor(feats.drop(columns=['class', 'Time Step', 'txId']).values, dtype=torch.float)
    edge_index = torch.tensor(edges.values, dtype=torch.long).T
    y = torch.tensor(feats['class'].values)
    time = torch.tensor(feats['Time Step'].values)
    data = Data(x=x, edge_index=edge_index, y=y, time=time)
    return data


train_data = get_data(df_train, df_edges_mapped_train)
test_data = get_data(df_test, df_edges_mapped_test)

In [23]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return torch.tensor(sample, dtype=torch.float32), torch.tensor(sample, dtype=torch.float32)

In [24]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim=50):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.Tanh(),
            nn.Linear(100, encoding_dim),
            nn.Tanh(),
            nn.Linear(encoding_dim, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, encoding_dim),
            nn.Tanh(),
            nn.Linear(encoding_dim, 100),
            nn.Tanh(),
            nn.Linear(100, input_dim),
            nn.ReLU()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [25]:
# Define the path to the saved model
model_path = 'C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/GNN_models/trained_models/ae_second_train.pth'
input_dim = test_data.x.shape[1]
# Create an instance of your model
model = Autoencoder(input_dim)

# Load the state_dict into the model
model.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode
model.eval()

# Convert input_data to a torch tensor if it's not already
train_input_data = train_data.x.clone().detach()
test_input_data = test_data.x.clone().detach()

# Pass the input_data through the encoder
train_latent_representation = model.encoder(train_input_data)
test_latent_representation = model.encoder(test_input_data)

In [26]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Assuming you have already obtained latent representations for your training data
# X_latent_train is the matrix of latent representations, y_train is the corresponding labels

# Initialize base classifiers
logistic_clf = LogisticRegression()
tree_clf = DecisionTreeClassifier()
random_forest_clf = RandomForestClassifier()
gradient_boosting_clf = GradientBoostingClassifier()

# Ensemble classifier combining base classifiers
ensemble_clf = VotingClassifier(estimators=[
    ('logistic', logistic_clf), 
    ('tree', tree_clf),
    ('random_forest', random_forest_clf),
    ('gradient_boosting', gradient_boosting_clf)
], voting='soft')

# Assuming train_latent_representation and test_latent_representation are tensors
train_input_latent = train_latent_representation.detach().numpy()
test_input_latent = test_latent_representation.detach().numpy()

# Train ensemble classifier
ensemble_clf.fit(train_input_latent, train_data.y)

# Predictions on training data
y_pred_train = ensemble_clf.predict(train_input_latent)

# Predictions on test data
y_pred_test = ensemble_clf.predict(test_input_latent)

# Evaluate performance
train_accuracy = accuracy_score(train_data.y, y_pred_train)
print("Accuracy on training set:", train_accuracy)

test_accuracy = accuracy_score(test_data.y, y_pred_test)
print("Accuracy on test set:", test_accuracy)


Accuracy on training set: 0.9853721266677383
Accuracy on test set: 0.9204373240961247


In [27]:
from sklearn.metrics import precision_recall_fscore_support

# Assuming you have already obtained predictions for your test data
# y_true is the true labels, y_pred is the predicted labels by the ensemble classifier

# Compute precision, recall, and F1 score for each class
precision, recall, f1_score, _ = precision_recall_fscore_support(test_data.y, y_pred_test)

# Print precision, recall, and F1 score for each class
for i in range(len(precision)):
    print(f"Class {i}: Precision={precision[i]}, Recall={recall[i]}, F1-score={f1_score[i]}")


Class 0: Precision=0.9537480063795853, Recall=0.6385477843032568, F1-score=0.7649504317236968
Class 1: Precision=0.9152054108216433, Recall=0.9921249151391718, F1-score=0.9521141442439247
